# Alpha–Phi Binary Explorer (v7)

* Fixes the **`list` vs numeric** bug by explicitly taking the last element of `hybrid_tuned` (`[-1]`).  
* Widens parameter ranges again:
  * `p`: 0.5 – 1.6  
  * `q`: 0.7 – 1.5  
  * `lam`: 0.01 – 0.08  
  * `gamma`: 0.02 – 0.07  
  * `phi_scale`: 0.7 – 1.1  
* Iteration depth doubled: `steps=60`, `cycles=15`.  
* Early‑exit once a < 1 % **or** a < 0.1 % hit is found, printing which threshold was crossed.  
* Still logs first 3 hits to `scan_log_v7.csv`.

Raise `mp.mp.dps` if you need more precision.

In [1]:
import mpmath as mp, csv, itertools, math
mp.mp.dps = 300
ALPHA = mp.mpf('7.2973525693e-3')
PHI   = (1 + mp.sqrt(5)) / 2
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]
def fib_word_real(n):
    bits=fib_word_bits(n)
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


In [2]:
def iterate_adapt(x0, steps, p, q, lam, gamma):
    traj=[x0]
    for _ in range(steps):
        eta = gamma*(x0/ALPHA-1)
        core=1/((10**(q+eta))*x0**p)
        x0 = core**lam
        traj.append(x0)
    return traj

In [3]:
def hybrid_tuned(x0, cycles, q, comp_p, phi_scale):
    traj=[x0]
    for _ in range(cycles):
        x0 = (phi_scale*PHI)/x0
        x0 = 1/(10**q * x0**comp_p)
        traj.append(x0)
    return traj

## Parameter sweep with bug‑fix (use last element of hybrid list)

In [4]:
# --- exhaustive-ish scan ---
hits=[]
with open('scan_log_v7.csv','w',newline='') as f:
    w=csv.writer(f); w.writerow(['p','q','lam','gamma','phi_scale','bits','rel_err'])
    space=itertools.product(
        (0.5,0.7,0.9,1.1,1.3,1.5),
        (0.7,0.9,1.1,1.3,1.5),
        (0.01,0.02,0.04,0.06,0.08),
        (0.02,0.04,0.05,0.07),
        (0.7,0.8,0.9,1.0,1.1),
        (512,1024)
    )
    for p,q,lam,gamma,phi_scale,bits in space:
        seed=fib_word_real(bits)
        x=iterate_adapt(seed,60,p,q,lam,gamma)[-1]
        x=hybrid_tuned(x,15,q,p,phi_scale)[-1]  # take last element
        rel=abs((x-ALPHA)/ALPHA)
        if rel<0.01:
            hits.append((rel,p,q,lam,gamma,phi_scale,bits))
            w.writerow([p,q,lam,gamma,phi_scale,bits,str(rel)])
            print('Hit <1%  ',rel,p,q,lam,gamma,phi_scale,bits)
            if rel<0.001 or len(hits)>=3:
                break
    else:
        print('No hit <1% in search.')


No hit <1% in search.
